#### import the things

In [1]:
import pandas as pd
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

idx = pd.IndexSlice

import scipy.stats
import data
raw_data_fol = os.path.join(data.__path__[0], "raw")

In [2]:
from src.data import data_sorting_tools as dst
%load_ext autoreload
%autoreload 2

In [304]:
def json_to_df(file, keys, get_media=True, return_dict=False):
    """creates dataframe from tweets based on which tweet keys you want.
    output dict is keyed on tweet ID"""
    with open(file, 'r') as f:
        doc = json.load(f)
    try:
        del doc['query']
    except:
        pass

    main_dict = {}
    for mainkey in doc.keys():
        temp = {}
        if ("retweeted_status" in doc[mainkey].keys())==True:
#             temp['retweet_of'] = doc[mainkey]["retweeted_status"]['id']
            root = doc[mainkey]["retweeted_status"]
        else:
            root = doc[mainkey]
        
        
        for k in keys:
            if type(k) is tuple:
                key = "{}_{}".format(k[0], k[1])
                temp[key] = root[k[0]][k[1]]
            else:
                temp[k] = root[k]   

        if ("full_text" in doc[mainkey].keys())==True:
            temp['full_text'] = doc[mainkey]["full_text"]
            
        if get_media==True:
            if "extended_entities" in doc[mainkey].keys():
                temp['media_count'] = len(doc[mainkey]['extended_entities']['media'])
                temp['media_types'] = doc[mainkey]['extended_entities']['media'][0]['type']

        tid = root['id']
        main_dict[tid] = temp

    df = pd.DataFrame(main_dict).T

    if return_dict == True:
        return df, main_dict
    else:
        return df

#### Understanding the key layout

In [4]:
query1_file = os.path.join(raw_data_fol, "main_search1_fixed.json")
with open(query1_file) as f:
    query1 = json.load(f)

In [5]:
query1['0'].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'extended_entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'matching_rules'])

In [12]:
query1['0']['created_at']

'Thu Feb 25 23:23:40 +0000 2021'

In [79]:
query1['0']['retweeted_status']

{'created_at': 'Wed Feb 24 14:43:50 +0000 2021',
 'id': 1364586840729268228,
 'id_str': '1364586840729268228',
 'text': '“Who gave you HIV?” https://t.co/XgAw6fnHdj',
 'display_text_range': [0, 19],
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 90162331,
  'id_str': '90162331',
  'name': 'TheKingofReads',
  'screen_name': 'TheKingofReads',
  'location': 'Atlanta, GA',
  'url': 'http://KingofReads.com',
  'description': 'Heavy is the head that wears the crown. Equal rights and equal opportunity reading. \n\nSupport me on Patreon\nhttps://www.patreon.com/kingofreads',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 23119,
  'friends_count': 572,
  'listed_count': 87,
  'favourites_count': 2575,

#### create dataframe from main dataset

In [20]:
keep_keys = ['quote_count', 'reply_count', 'retweet_count', 'favorite_count', ('user','id'), 
                ('user','followers_count'), ('user','verified'), 'text', 'lang']

In [305]:
all_dfs = []
for f in os.listdir(raw_data_fol):
    if ("main" in f) and (".json" in f):
        d = json_to_df(os.path.join(raw_data_fol, f), keep_keys, get_media=True, return_dict=False)
        all_dfs.append(d)
#         print(f, len(d))
        
df = pd.concat(all_dfs)

In [306]:
df.head()

,quote_count,reply_count,retweet_count,favorite_count,user_id,user_followers_count,user_verified,text,lang,full_text,media_count,media_types
1363920409696079876,1,1,11,37,41335459,3344,False,Published today: analysis of @Prime_UCL - over...,en,Published today: analysis of @Prime_UCL - over...,NaN,NaN
1362099498563088390,7,14,336,6174,19177501,279378,True,There could be 123 000 to 293 000 additional H...,en,There could be 123 000 to 293 000 additional H...,NaN,NaN
1364155919119110151,43,19,168,236,175055258,41040,False,"Even your HIV status, don't declare to potenti...",en,NaN,NaN,NaN
1333476068192366593,0,2,3,16,2152363202,6070,False,"Teen pregnancy is high, HIV infection rate is ...",en,NaN,NaN,NaN
1364161232270487553,0,0,0,0,1331332400836726785,62,False,Even though it was a charity. Stevens thought...,en,Even though it was a charity. Stevens thought...,NaN,NaN


In [307]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11307 entries, 1363920409696079876 to 1364931681220653057
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   quote_count           11307 non-null  object
 1   reply_count           11307 non-null  object
 2   retweet_count         11307 non-null  object
 3   favorite_count        11307 non-null  object
 4   user_id               11307 non-null  object
 5   user_followers_count  11307 non-null  object
 6   user_verified         11307 non-null  object
 7   text                  11307 non-null  object
 8   lang                  11307 non-null  object
 9   full_text             8459 non-null   object
 10  media_count           308 non-null    object
 11  media_types           308 non-null    object
dtypes: object(12)
memory usage: 1.1+ MB


In [308]:
df['media_count'] = df['media_count'].fillna(0)
df['media_types'] = df['media_types'].fillna('none')

In [309]:
df = df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11307 entries, 1363920409696079876 to 1364931681220653057
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           11307 non-null  Int64  
 1   reply_count           11307 non-null  Int64  
 2   retweet_count         11307 non-null  Int64  
 3   favorite_count        11307 non-null  Int64  
 4   user_id               11307 non-null  Int64  
 5   user_followers_count  11307 non-null  Int64  
 6   user_verified         11307 non-null  boolean
 7   text                  11307 non-null  string 
 8   lang                  11307 non-null  string 
 9   full_text             8459 non-null   string 
 10  media_count           11307 non-null  Int64  
 11  media_types           11307 non-null  string 
dtypes: Int64(7), boolean(1), string(4)
memory usage: 1.1 MB


In [310]:
df.set_index('user_id', drop=True, append=True, inplace=True)
df.index.rename(['tid', 'uid'], inplace=True)

In [311]:
df = df.assign(same_text = lambda x: x['full_text'] == x['text'])

In [312]:
df.head()

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,full_text,media_count,media_types,same_text
tid,uid,,,,,,,,,,,,
1363920409696079876,41335459,1,1,11,37,3344,False,Published today: analysis of @Prime_UCL - over...,en,Published today: analysis of @Prime_UCL - over...,0,none,False
1362099498563088390,19177501,7,14,336,6174,279378,True,There could be 123 000 to 293 000 additional H...,en,There could be 123 000 to 293 000 additional H...,0,none,False
1364155919119110151,175055258,43,19,168,236,41040,False,"Even your HIV status, don't declare to potenti...",en,<NA>,0,none,<NA>
1333476068192366593,2152363202,0,2,3,16,6070,False,"Teen pregnancy is high, HIV infection rate is ...",en,<NA>,0,none,<NA>
1364161232270487553,1331332400836726785,0,0,0,0,62,False,Even though it was a charity. Stevens thought...,en,Even though it was a charity. Stevens thought...,0,none,False


In [313]:
df[['text', 'full_text', 'same_text']]

,,text,full_text,same_text
tid,uid,,,
1363920409696079876,41335459,Published today: analysis of @Prime_UCL - over...,Published today: analysis of @Prime_UCL - over...,False
1362099498563088390,19177501,There could be 123 000 to 293 000 additional H...,There could be 123 000 to 293 000 additional H...,False
1364155919119110151,175055258,"Even your HIV status, don't declare to potenti...",<NA>,<NA>
1333476068192366593,2152363202,"Teen pregnancy is high, HIV infection rate is ...",<NA>,<NA>
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...,Even though it was a charity. Stevens thought...,False
...,...,...,...,...
1364366548417318913,533233011,In the 55th plenary meeting of the 75th #UNGA ...,In the 55th plenary meeting of the 75th #UNGA ...,False
1364927797349801984,77200294,We're delighted to have worked with Lord Fowle...,We're delighted to have worked with Lord Fowle...,False
1364931957977735172,1327999757634199552,with the LGBT poll thing yesterday I didn't se...,<NA>,<NA>


In [314]:
df[df.same_text==True][['text', 'full_text', 'same_text']]

,,text,full_text,same_text
tid,uid,,,


In [315]:
df.iloc[0]['text']

'Published today: analysis of @Prime_UCL - over 50% of women with HIV aged 45-60 reported moderate/extreme pain. Inc… https://t.co/lrGUY7PYRx'

In [316]:
df.iloc[0]['full_text']

'Published today: analysis of @Prime_UCL - over 50% of women with HIV aged 45-60 reported moderate/extreme pain. Increased prevalence in perimenopause. Pain assoc with insomnia + depression highlighting importance of eliciting pain history. Read more: https://t.co/woqgA0MGku https://t.co/vPNi0ORxse'

In [317]:
df.iloc[2]['text']

"Even your HIV status, don't declare to potential partners if you don't feel safe."

In [318]:
df['full_text'].fillna(df['text'], inplace=True)

In [319]:
df[['text', 'full_text', 'same_text']]

,,text,full_text,same_text
tid,uid,,,
1363920409696079876,41335459,Published today: analysis of @Prime_UCL - over...,Published today: analysis of @Prime_UCL - over...,False
1362099498563088390,19177501,There could be 123 000 to 293 000 additional H...,There could be 123 000 to 293 000 additional H...,False
1364155919119110151,175055258,"Even your HIV status, don't declare to potenti...","Even your HIV status, don't declare to potenti...",<NA>
1333476068192366593,2152363202,"Teen pregnancy is high, HIV infection rate is ...","Teen pregnancy is high, HIV infection rate is ...",<NA>
1364161232270487553,1331332400836726785,Even though it was a charity. Stevens thought...,Even though it was a charity. Stevens thought...,False
...,...,...,...,...
1364366548417318913,533233011,In the 55th plenary meeting of the 75th #UNGA ...,In the 55th plenary meeting of the 75th #UNGA ...,False
1364927797349801984,77200294,We're delighted to have worked with Lord Fowle...,We're delighted to have worked with Lord Fowle...,False
1364931957977735172,1327999757634199552,with the LGBT poll thing yesterday I didn't se...,with the LGBT poll thing yesterday I didn't se...,<NA>


In [320]:
df.drop(columns=['text', 'same_text'], inplace=True)

In [321]:
df.lang.unique()

<StringArray>
['en', 'no', 'tl', 'fr', 'da', 'in']
Length: 6, dtype: string

In [322]:
df[df.lang!='en'].full_text.values

<StringArray>
[                                                                                                                                                                                                                                                          '@Nigeriangod_ God forbid o\nI no get HIV😂',
                                                                                                                                                                                                                                               'Free hiv testing\n\nDm lng po https://t.co/ZLQsG6imfs',
                                                                                                                                                                                                                    '@ssfarouk @anuja_java @AST_TYFCOP @ssfarouk IL-2RAs in our HIV patients for sure',
                                                                                                  

keep the first 3, but drop the rest as they're not in english (still don't understand how that got through my filter)

In [323]:
df[df.lang!='en'][3:].index

MultiIndex([(1361868392337084417,          2886959544),
            (1361810081067667457,          2888956394),
            (1361722173279272961, 1289987297950068736),
            (1187203565309153280, 1176579988516032514)],
           names=['tid', 'uid'])

In [324]:
df.drop(index=df[df.lang!='en'][3:].index)

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,lang,full_text,media_count,media_types
tid,uid,,,,,,,,,,
1363920409696079876,41335459,1,1,11,37,3344,False,en,Published today: analysis of @Prime_UCL - over...,0,none
1362099498563088390,19177501,7,14,336,6174,279378,True,en,There could be 123 000 to 293 000 additional H...,0,none
1364155919119110151,175055258,43,19,168,236,41040,False,en,"Even your HIV status, don't declare to potenti...",0,none
1333476068192366593,2152363202,0,2,3,16,6070,False,en,"Teen pregnancy is high, HIV infection rate is ...",0,none
1364161232270487553,1331332400836726785,0,0,0,0,62,False,en,Even though it was a charity. Stevens thought...,0,none
...,...,...,...,...,...,...,...,...,...,...,...
1364366548417318913,533233011,1,1,12,27,14571,True,en,In the 55th plenary meeting of the 75th #UNGA ...,0,none
1364927797349801984,77200294,1,0,9,28,22961,True,en,We're delighted to have worked with Lord Fowle...,0,none
1364931957977735172,1327999757634199552,0,1,0,0,17,False,en,with the LGBT poll thing yesterday I didn't se...,0,none


In [325]:
df.groupby('tid').count().sort_values('quote_count', ascending=False)

,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,lang,full_text,media_count,media_types
tid,,,,,,,,,,
1362871966722633736,20,20,20,20,20,20,20,20,20,20
1360551795098079233,19,19,19,19,19,19,19,19,19,19
1363149914222432258,18,18,18,18,18,18,18,18,18,18
1358792705728413696,17,17,17,17,17,17,17,17,17,17
1363111160925409286,17,17,17,17,17,17,17,17,17,17
...,...,...,...,...,...,...,...,...,...,...
1362966214385967104,1,1,1,1,1,1,1,1,1,1
1362963823389507589,1,1,1,1,1,1,1,1,1,1
1362959163089969152,1,1,1,1,1,1,1,1,1,1


In [326]:
duplicates = df[df.index.get_level_values('tid').duplicated(keep='first')].index

In [327]:
df.drop(index=duplicates).groupby('tid').count().sort_values('quote_count', ascending=False)

,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,lang,full_text,media_count,media_types
tid,,,,,,,,,,
526021606431526912,1,1,1,1,1,1,1,1,1,1
1364020582665306118,1,1,1,1,1,1,1,1,1,1
1364022680140218369,1,1,1,1,1,1,1,1,1,1
1364022138269814785,1,1,1,1,1,1,1,1,1,1
1364021986519773187,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
1362672149761200128,1,1,1,1,1,1,1,1,1,1
1362664604040527874,1,1,1,1,1,1,1,1,1,1
1362657908585091073,1,1,1,1,1,1,1,1,1,1


In [328]:
df = df.drop(index=duplicates)

In [329]:
df.shape

(7095, 10)

In [330]:
df.head()

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,lang,full_text,media_count,media_types
tid,uid,,,,,,,,,,
1333476068192366593,2152363202,0,2,3,16,6070,False,en,"Teen pregnancy is high, HIV infection rate is ...",0,none
1364161232270487553,1331332400836726785,0,0,0,0,62,False,en,Even though it was a charity. Stevens thought...,0,none
1364161201291153414,150301804,0,0,0,0,1044,False,en,SCORA Standing Committee on Sexual &amp; Repro...,0,none
1364161184505737217,1317091162159747075,0,0,0,0,15,False,en,"many females are HIV+, we wish you well, bless...",0,none
1363439109948149760,1353391442408792066,0,0,2,24,184,False,en,@TheRustler83 Yep. Imagine if the government w...,0,none


In [331]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7095 entries, (1333476068192366593, 2152363202) to (1364931681220653057, 1337754019163496450)
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           7095 non-null   Int64  
 1   reply_count           7095 non-null   Int64  
 2   retweet_count         7095 non-null   Int64  
 3   favorite_count        7095 non-null   Int64  
 4   user_followers_count  7095 non-null   Int64  
 5   user_verified         7095 non-null   boolean
 6   lang                  7095 non-null   string 
 7   full_text             7095 non-null   string 
 8   media_count           7095 non-null   Int64  
 9   media_types           7095 non-null   string 
dtypes: Int64(6), boolean(1), string(3)
memory usage: 1.1 MB


this leaves me with only ~7100 tweets out of the original 20000 that I pulled. :( Maybe I can salvge some from a previous pull

In [332]:
#sort through previously collected tweets as well
all_dfs = []
for f in os.listdir(os.path.join(raw_data_fol, 'second_main_search')):
    d = json_to_df(os.path.join(raw_data_fol, 'second_main_search', f), keep_keys, get_media=True, return_dict=False)
    all_dfs.append(d)
        
df2 = pd.concat(all_dfs)

In [333]:
df2.head()

,quote_count,reply_count,retweet_count,favorite_count,user_id,user_followers_count,user_verified,text,lang,media_count,media_types
1363949890598969344,0,0,0,0,1148573841020428290,10,False,大昔の性病とされてきた梅毒に感染する男女が急増中です。HIVやクラミジア、淋病と違い、コンド...,ja,NaN,NaN
1363949864866971654,0,1,0,0,1101131404929392641,9086,False,@isolapine @GrandjeanBenoit a nan mais bien su...,fr,NaN,NaN
1363949818079375360,0,1,0,18,1968806107,12,False,@N65Michelle @emzanotti @MeghanMcCain I agree-...,en,NaN,NaN
1363896551458934784,16,6,470,2430,227505716,23048,False,# Esclarecimentos e informações que toda Pesso...,pt,NaN,NaN
1363496045515321345,0,0,1,5,624216351,4094,False,Great 🧵 on #visceralLeishmaniasis important to...,en,NaN,NaN


In [334]:
df2['media_count'] = df2['media_count'].fillna(0)
df2['media_types'] = df2['media_types'].fillna('none')

In [335]:
df2 = df2.convert_dtypes()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3803 entries, 1363949890598969344 to 1363851559612739593
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           3803 non-null   Int64  
 1   reply_count           3803 non-null   Int64  
 2   retweet_count         3803 non-null   Int64  
 3   favorite_count        3803 non-null   Int64  
 4   user_id               3803 non-null   Int64  
 5   user_followers_count  3803 non-null   Int64  
 6   user_verified         3803 non-null   boolean
 7   text                  3803 non-null   string 
 8   lang                  3803 non-null   string 
 9   media_count           3803 non-null   Int64  
 10  media_types           3803 non-null   string 
dtypes: Int64(7), boolean(1), string(3)
memory usage: 360.2 KB


In [336]:
df2.set_index('user_id', drop=True, append=True, inplace=True)
df2.index.rename(['tid', 'uid'], inplace=True)

this data pull had a lot of non-english tweets

In [337]:
df2.lang.unique()

<StringArray>
[ 'ja',  'fr',  'en',  'pt',  'it',  'vi',  'tr',  'es', 'und',  'sv',  'de',
  'fa',  'in',  'et',  'mr',  'pl',  'cs',  'ru',  'ar',  'ro',  'zh',  'tl',
  'da',  'hi',  'hu',  'th',  'ht',  'no',  'nl',  'iw',  'sl',  'el',  'fi',
  'my',  'ca',  'gu',  'ta']
Length: 37, dtype: string

In [338]:
non_eng = df2[df2.lang!='en'].index

In [339]:
df2.drop(index=non_eng, inplace=True)

In [340]:
df2.head()

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types
tid,uid,,,,,,,,,,
1363949818079375360,1968806107,0,1,0,18,12,False,@N65Michelle @emzanotti @MeghanMcCain I agree-...,en,0,none
1363496045515321345,624216351,0,0,1,5,4094,False,Great 🧵 on #visceralLeishmaniasis important to...,en,0,none
1363946125091348482,31583882,3,1,9,21,471789,True,"Join us for the HIV Stigma and Faith Summit, W...",en,0,none
1363888791061221386,1957125536,3,14,68,401,73230,False,Hard Left: We must pursue ZERO Covid policy - ...,en,0,none
1363949692288004103,308788980,0,0,0,1,548,False,So he says what? This is the guy that was in c...,en,0,none


I also didn't fix the truncated tweets for this data pull, so I'll have to find them another way. but first.

In [341]:
duplicates = df2[df2.index.get_level_values('tid').duplicated(keep='first')].index

In [342]:
df2.drop(index=duplicates, inplace=True)

In [343]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1711 entries, (1363949818079375360, 1968806107) to (1363562391460597765, 2463473684)
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           1711 non-null   Int64  
 1   reply_count           1711 non-null   Int64  
 2   retweet_count         1711 non-null   Int64  
 3   favorite_count        1711 non-null   Int64  
 4   user_followers_count  1711 non-null   Int64  
 5   user_verified         1711 non-null   boolean
 6   text                  1711 non-null   string 
 7   lang                  1711 non-null   string 
 8   media_count           1711 non-null   Int64  
 9   media_types           1711 non-null   string 
dtypes: Int64(6), boolean(1), string(3)
memory usage: 340.1 KB


In [344]:
df2.iloc[0].text

'@N65Michelle @emzanotti @MeghanMcCain I agree-I was teaching HIV/AIDS prevention classes way back when and used Dr… https://t.co/rriyIgjSZ0'

In [346]:
'…' in df2.iloc[0].text

True

In [347]:
is_trunc = []
for ind in df2.index:
    is_trunc.append('…' in df2.loc[ind]['text'])

In [348]:
df2['is_trunc'] = is_trunc

In [349]:
df2[df2.is_trunc==False]

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types,is_trunc
tid,uid,,,,,,,,,,,
1363949568040312835,1010757472238342145,0,0,0,2,179,False,"#RT @pozmagazine: RT @fundacionvihda: Today, #...",en,0,none,False
1363949506572808199,1310327055209369601,0,0,0,2,291,False,@Sham_WRcH @BBC3CR @SAHAR_hiv @CNWLNHS @MKCHS ...,en,0,none,False
1363949279010766848,294283011,0,0,0,1,1255,False,"#Eastenders My guess is, Isaac has the HIV/Hep...",en,0,none,False
1339524975867400193,1258520602102530048,1,16,35,71,2817,False,HIV/AIDS vaccine is Launched Condom companie...,en,1,photo,False
1363948780010287112,1232584407065747461,0,0,0,2,5038,False,i asked dan for his status on HIV and oh god d...,en,0,none,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1363851940791078913,1281625078963122177,0,0,0,0,7,False,Using AI to reduce the risk of HIV – Harvard G...,en,0,none,False
1363851935145742340,1330084058798444547,0,0,0,0,10,False,"L need to say this ,,,,,,, do you know that yo...",en,0,none,False
1363851681021136900,2234067829,0,0,1,0,2108,False,Sindh to screen fresh school students for tube...,en,1,photo,False


that's barely any extra tweets. It seems like it's worth getting the full text of the truncated tweets from this set.

In [350]:
trunc_ind = df2[df2.is_trunc==True].index

In [351]:
from src.data import twitter_api_access
api = twitter_api_access.access_api()

In [174]:
fix_dict = {}
for ind in trunc_ind:
    try:
        status = api.get_status(ind[0], tweet_mode="extended")
        try:
            full_text = status.retweeted_status.full_text
        except AttributeError:
            full_text = status.full_text
        fix_dict[ind] = full_text
    except:
        pass

Rate limit reached. Sleeping for: 721


In [352]:
list(fix_dict.keys())[0]

(1363949818079375360, 1968806107)

In [353]:
fix_dict[(1363949818079375360, 1968806107)]

'@N65Michelle @emzanotti @MeghanMcCain I agree-I was teaching HIV/AIDS prevention classes way back when and used Dr Fauci and the NIH for much of my resources.'

In [354]:
df2.loc[(1363949818079375360, 1968806107)].text

'@N65Michelle @emzanotti @MeghanMcCain I agree-I was teaching HIV/AIDS prevention classes way back when and used Dr… https://t.co/rriyIgjSZ0'

In [355]:
df2_full_text = pd.DataFrame(fix_dict.values(), index=fix_dict.keys())

In [356]:
df2_full_text.index.rename(['tid', 'uid'], inplace=True)

In [357]:
df2_full_text.head()

,,0
tid,uid,
1363949818079375360,1968806107,@N65Michelle @emzanotti @MeghanMcCain I agree-...
1363496045515321345,624216351,Great 🧵 on #visceralLeishmaniasis important to...
1363946125091348482,31583882,"Join us for the HIV Stigma and Faith Summit, W..."
1363949692288004103,308788980,So he says what? This is the guy that was in c...
1363946923468726276,49478637,Huge thanks to @SophiaForum and partners for t...


In [358]:
df2 = df2.merge(df2_full_text, left_index=True, right_index=True, how='outer')

In [359]:
df2[0].fillna(df2['text'], inplace=True)

In [360]:
df2.head()

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types,is_trunc,0
tid,uid,,,,,,,,,,,,
1079473214315741186,485964552,280,43,15308,8350,25894,False,No AIDS🚫 No Chlamydia🚫 No Crabs🚫 No Trichomoni...,en,0,none,True,No AIDS🚫\nNo Chlamydia🚫\nNo Crabs🚫\nNo Trichom...
1135529653475696641,211106246,1,2,5,5,2289,False,LOL. So a clinic is looking for premium sperm ...,en,0,none,True,LOL. So a clinic is looking for premium sperm ...
1151121776061165568,1393157736,13,18,164,314,928,False,Just incase anyone forgot how awful Piers Morg...,en,0,none,True,Just incase anyone forgot how awful Piers Morg...
1195090950776926222,114985093,153,10,2568,7039,30396,False,Stop fucking multiple women in the raw bro. Wh...,en,0,none,True,Stop fucking multiple women in the raw bro. Wh...
1236024261611003904,148477475,0,0,29,32,53559,True,March 10 is National Women &amp; Girls HIV/AID...,en,0,none,True,March 10 is National Women &amp; Girls HIV/AID...


In [361]:
df2.rename(columns={0:'full_text'}, inplace=True)

In [362]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1711 entries, (1079473214315741186, 485964552) to (1363949818079375360, 1968806107)
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           1711 non-null   Int64  
 1   reply_count           1711 non-null   Int64  
 2   retweet_count         1711 non-null   Int64  
 3   favorite_count        1711 non-null   Int64  
 4   user_followers_count  1711 non-null   Int64  
 5   user_verified         1711 non-null   boolean
 6   text                  1711 non-null   string 
 7   lang                  1711 non-null   string 
 8   media_count           1711 non-null   Int64  
 9   media_types           1711 non-null   string 
 10  is_trunc              1711 non-null   bool   
 11  full_text             1711 non-null   object 
dtypes: Int64(6), bool(1), boolean(1), object(1), string(3)
memory usage: 299.8+ KB


In [363]:
df.columns

Index(['quote_count', 'reply_count', 'retweet_count', 'favorite_count',
       'user_followers_count', 'user_verified', 'lang', 'full_text',
       'media_count', 'media_types'],
      dtype='object')

In [364]:
df2.drop(columns='text', inplace=True)
df2.drop(columns='is_trunc', inplace=True)

In [365]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1711 entries, (1079473214315741186, 485964552) to (1363949818079375360, 1968806107)
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           1711 non-null   Int64  
 1   reply_count           1711 non-null   Int64  
 2   retweet_count         1711 non-null   Int64  
 3   favorite_count        1711 non-null   Int64  
 4   user_followers_count  1711 non-null   Int64  
 5   user_verified         1711 non-null   boolean
 6   lang                  1711 non-null   string 
 7   media_count           1711 non-null   Int64  
 8   media_types           1711 non-null   string 
 9   full_text             1711 non-null   object 
dtypes: Int64(6), boolean(1), object(1), string(2)
memory usage: 284.7+ KB


In [366]:
full_df = pd.concat([df, df2])

In [367]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8806 entries, (1333476068192366593, 2152363202) to (1363949818079375360, 1968806107)
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           8806 non-null   Int64  
 1   reply_count           8806 non-null   Int64  
 2   retweet_count         8806 non-null   Int64  
 3   favorite_count        8806 non-null   Int64  
 4   user_followers_count  8806 non-null   Int64  
 5   user_verified         8806 non-null   boolean
 6   lang                  8806 non-null   string 
 7   full_text             8806 non-null   object 
 8   media_count           8806 non-null   Int64  
 9   media_types           8806 non-null   string 
dtypes: Int64(6), boolean(1), object(1), string(2)
memory usage: 1.3+ MB


In [368]:
duplicates = full_df[full_df.index.get_level_values('tid').duplicated(keep='first')].index
len(duplicates)

572

In [369]:
full_df.drop(index=duplicates, inplace=True)

In [370]:
full_df.shape

(7662, 10)

In [371]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 7662 entries, (1333476068192366593, 2152363202) to (1363949692288004103, 308788980)
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           7662 non-null   Int64  
 1   reply_count           7662 non-null   Int64  
 2   retweet_count         7662 non-null   Int64  
 3   favorite_count        7662 non-null   Int64  
 4   user_followers_count  7662 non-null   Int64  
 5   user_verified         7662 non-null   boolean
 6   lang                  7662 non-null   string 
 7   full_text             7662 non-null   object 
 8   media_count           7662 non-null   Int64  
 9   media_types           7662 non-null   string 
dtypes: Int64(6), boolean(1), object(1), string(2)
memory usage: 1.2+ MB


all of that for another ~600 tweets. 

In [372]:
full_df.drop(columns='lang', inplace=True)

In [373]:
full_df.head()

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,full_text,media_count,media_types
tid,uid,,,,,,,,,
1333476068192366593,2152363202,0,2,3,16,6070,False,"Teen pregnancy is high, HIV infection rate is ...",0,none
1364161232270487553,1331332400836726785,0,0,0,0,62,False,Even though it was a charity. Stevens thought...,0,none
1364161201291153414,150301804,0,0,0,0,1044,False,SCORA\nStanding Committee on Sexual &amp; Repr...,0,none
1364161184505737217,1317091162159747075,0,0,0,0,15,False,"many females are HIV+, we wish you well, bless...",0,none
1363439109948149760,1353391442408792066,0,0,2,24,184,False,@TheRustler83 Yep. Imagine if the government w...,0,none


In [374]:
#save the dataframe as a pickle file to be loaded into other notebooks
pd.to_pickle(full_df, os.path.join(data.__path__[0], "compiled", "full_dataset_cleaned.pkl"))